# 第七章：数据清洗与预处理 Data Cleaning and Preparation

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 7.1 处理缺失值 Handling Missing Data

NaN：Not a Number

NA：Not available

None

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

In [ ]:
string_data.isnull()

In [ ]:
from numpy import nan as NA
string_data[0] = None
string_data[1] = NA
string_data

In [ ]:
string_data.isnull()

### 7.1.1 过滤缺失值 Filtering Out Missing Data

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data

In [ ]:
data.dropna()

In [ ]:
data

等价于下面这个

In [ ]:
data[data.notnull()] 

DataFrame对象

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
data

data.dropna()删除包含缺失值的行或列

In [ ]:
cleaned = data.dropna()
data

In [ ]:
cleaned

how='all'，表示删除全部为缺失值的行

In [ ]:
data.dropna(how='all')

如果想删除全为缺失值的列

In [ ]:
data[4] = NA
data

传入参数axis=1

In [ ]:
data.dropna(axis=1, how='all')

如果想删除包含两个缺失值以上的行

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

In [ ]:
df.dropna()

可以设置thresh=2

In [ ]:
df.dropna(thresh=2)

### 7.1.2 补全缺失值 Filling In Missing Data

fillna()可以使用常数来代替缺失值

In [ ]:
df.fillna(0)

填充缺失值是使用字典，可以指定不同的列用不同的值填充

In [ ]:
df.fillna({1: 0.5, 2: 0})

In [ ]:
df

fillna返回的是一个新的对象，但你也可以修改已经存在的对象

In [ ]:
_ = df.fillna(0, inplace=True)
df

生成一个例子

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

method='ffill'，填充方法，这个默认是前项填充

In [ ]:
df.fillna(method='ffill')

参数limit，最大填充范围

In [ ]:
df.fillna(method='ffill', limit=2)

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data

In [ ]:
data.fillna(data.mean())

## 7.2 数据转换 Data Transformation

In [4]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

### 7.2.1 删除重复值 Removing Duplicates

由于各种原因，DataFrame会出现重复行

In [5]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


判断该行之前是否出现过

In [3]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

删除重复的行

In [7]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


对于某列的重复值

In [8]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


可以指定列名

In [9]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


默认重复的值，默认保留第一个检测到的

传入 keep='last'，将会返回后一个

In [10]:
data.drop_duplicates(['k2'], keep='last')

,k1,k2,v1
1,two,1,1
2,one,2,2
4,one,3,4
6,two,4,6


### 7.2.2 使用函数或映射进行数据转换 Transforming Data Using a Function or Mapping

In [27]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


想要对食品中肉的种类做个标签

In [28]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

首先将字母统一为小写

In [29]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

.map()，按照字典的指定进行匹配

In [30]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


也可以写成下面这种形式

In [36]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [37]:
data['food'].map(lambda x: print(x))

bacon
pulled pork
bacon
Pastrami
corned beef
Bacon
pastrami
honey ham
nova lox


0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
Name: food, dtype: object

In [38]:
meat_to_animal['bacon']

'pig'

### 7.2.3 替代值 Replacing Values

replace也可以用于替换

In [17]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

-999很可能是缺失值的标识

In [18]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

一次替代多个值

In [19]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

替换为不同的值，用列表

In [20]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

或者用字典的形式传递参数

In [21]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.2.4 重命名轴索引 Renaming Axis Indexes

In [39]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


先定义一个匿名函数，取出0-3，全大写

In [40]:
transform = lambda x: x[:4].upper()
transform

<function __main__.<lambda>(x)>

data的index属性中取出前四个，并执行upper函数

In [41]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

让原来的index，等于转为大写处理之后的，可以看到index被替换掉了

In [42]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


或者使用rename方法

In [43]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


可以用字典指定，替换方案

In [44]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


想要改变原有变量，传入参数inplace=True

In [45]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 离散化和分箱 Discretization and Binning

也就是分组

In [46]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

分为18~25,26~35,36~60，61~100

In [47]:
bins = [18, 25, 35, 60, 100]

 pd.cut(ages, bins)进行分组
 
 返回的对象是一个特殊的对象

In [48]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

查看里面的属性，12个数字对应分组标签

如20属于第0类

In [49]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

相当于x轴

In [50]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

统计一下每个里面的个数

In [51]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

上面的开区间与闭区间可以自己设置

传入right=False可以变为左闭右开区间

In [52]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

可以给分类价格名称

In [53]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

数据均匀分成四份

In [54]:
data = np.random.rand(20)
data

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536, 0.7477, 0.9613, 0.0084, 0.1064, 0.2987, 0.6564,
       0.8098, 0.8722, 0.9646, 0.7237])

 precision=2将十进制精度限制为两位

In [55]:
pd.cut(data, 4, precision=2)

[(0.73, 0.96], (0.25, 0.49], (0.0074, 0.25], (0.0074, 0.25], (0.49, 0.73], ..., (0.49, 0.73], (0.73, 0.96], (0.73, 0.96], (0.73, 0.96], (0.49, 0.73]]
Length: 20
Categories (4, interval[float64]): [(0.0074, 0.25] < (0.25, 0.49] < (0.49, 0.73] < (0.73, 0.96]]

qcut基于样本的分为数分割

In [56]:
data = np.random.randn(1000)  # Normally distributed
data

array([ 1.3529,  0.8864, -2.0016, -0.3718,  1.669 , -0.4386, -0.5397,
        0.477 ,  3.2489, -1.0212, -0.5771,  0.1241,  0.3026,  0.5238,
        0.0009,  1.3438, -0.7135, -0.8312, -2.3702, -1.8608, -0.8608,
        0.5601, -1.2659,  0.1198, -1.0635,  0.3329, -2.3594, -0.1995,
       -1.542 , -0.9707, -1.307 ,  0.2863,  0.378 , -0.7539,  0.3313,
        1.3497,  0.0699,  0.2467, -0.0119,  1.0048,  1.3272, -0.9193,
       -1.5491,  0.0222,  0.7584, -0.6605,  0.8626, -0.01  ,  0.05  ,
        0.6702,  0.853 , -0.9559, -0.0235, -2.3042, -0.6525, -1.2183,
       -1.3326,  1.0746,  0.7236,  0.69  ,  1.0015, -0.5031, -0.6223,
       -0.9212, -0.7262,  0.2229,  0.0513, -1.1577,  0.8167,  0.4336,
        1.0107,  1.8249, -0.9975,  0.8506, -0.1316,  0.9124,  0.1882,
        2.1695, -0.1149,  2.0037,  0.0296,  0.7953,  0.1181, -0.7485,
        0.585 ,  0.1527, -1.5657, -0.5625, -0.0327, -0.929 , -0.4826,
       -0.0363,  1.0954,  0.9809, -0.5895,  1.5817, -0.5287,  0.457 ,
        0.93  , -1.5

In [57]:
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

[(0.626, 3.928], (0.626, 3.928], (-2.9499999999999997, -0.691], (-0.691, -0.0171], (0.626, 3.928], ..., (-0.0171, 0.626], (0.626, 3.928], (-0.691, -0.0171], (-0.691, -0.0171], (-0.0171, 0.626]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.691] < (-0.691, -0.0171] < (-0.0171, 0.626] < (0.626, 3.928]]

每个范围都是250个

In [58]:
pd.value_counts(cats)

(0.626, 3.928]                   250
(-0.0171, 0.626]                 250
(-0.691, -0.0171]                250
(-2.9499999999999997, -0.691]    250
dtype: int64

可以传入自定义分位数，0-1之间的数组

In [59]:
cats = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats

[(1.297, 3.928], (-0.0171, 1.297], (-2.9499999999999997, -1.191], (-1.191, -0.0171], (1.297, 3.928], ..., (-0.0171, 1.297], (-0.0171, 1.297], (-1.191, -0.0171], (-1.191, -0.0171], (-0.0171, 1.297]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -1.191] < (-1.191, -0.0171] < (-0.0171, 1.297] < (1.297, 3.928]]

In [61]:
pd.value_counts(cats)

(-0.0171, 1.297]                 400
(-1.191, -0.0171]                400
(1.297, 3.928]                   100
(-2.9499999999999997, -1.191]    100
dtype: int64

### 7.2.6 检测和过滤异常值 Detecting and Filtering Outliers

下面是一个具有正态分布数据的DataFrame

In [62]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.021425,-0.005165,-0.055670,0.044107
std,1.008404,0.995532,0.994484,0.996884
min,-3.184377,-3.745356,-3.428254,-3.645860
25%,-0.628122,-0.699383,-0.747478,-0.599807
50%,-0.016127,-0.029924,-0.091364,0.043663
75%,0.690847,0.694459,0.620197,0.740562
max,3.525865,2.735527,3.366626,2.653656


假设想找出一列中绝对值大于三的值

In [63]:
col = data[2]
col[np.abs(col) > 3]

269   -3.428254
646    3.366626
Name: 2, dtype: float64

选出所有绝对值大于3的，使用any方法

In [65]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
52,-0.025907,-3.399312,-0.974657,-0.685312
71,3.260383,0.963301,1.201206,-1.852001
147,-0.196713,-3.745356,-1.520113,-0.346839
246,-3.056990,1.918403,-0.578828,1.847446
269,0.326045,0.425384,-3.428254,-0.296336
333,-3.184377,1.369891,-1.074833,-0.089937
554,0.208011,-0.150923,-0.362528,-3.548824
646,0.193299,1.397822,3.366626,-2.372214
793,3.525865,0.283070,0.544635,0.462204
813,-0.450721,-0.080332,0.599947,-3.645860


绝对值大于3的数，变为-3或者3

np.sign(data)，数值的符号，-1或者1

In [66]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.020880,-0.004021,-0.055608,0.045301
std,1.005160,0.991718,0.991924,0.992863
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.628122,-0.699383,-0.747478,-0.599807
50%,-0.016127,-0.029924,-0.091364,0.043663
75%,0.690847,0.694459,0.620197,0.740562
max,3.000000,2.735527,3.000000,2.653656


np.sign(data)

In [67]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,1.0
1,-1.0,-1.0,-1.0,-1.0
2,1.0,1.0,1.0,-1.0
3,1.0,1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,-1.0


### 7.2.7 置换和随机抽样 Permutation and Random Sampling

In [76]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


生成一个长度为5的随机数数组

In [77]:
sampler = np.random.permutation(5)
sampler

array([1, 0, 4, 2, 3])

take函数，按索引排序

In [78]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
2,8,9,10,11
3,12,13,14,15


生成一个随机样本，不可重复

In [85]:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
1,4,5,6,7


随机抽样，可重复的话replace=True

In [88]:
choices = pd.Series([5, 7, -1, 6, 4])
choices

0    5
1    7
2   -1
3    6
4    4
dtype: int64

In [89]:
draws = choices.sample(n=10, replace=True)
draws

2   -1
1    7
3    6
3    6
1    7
4    4
4    4
4    4
2   -1
0    5
dtype: int64

### 7.2.8 计算指标/虚拟变量 Computing Indicator/Dummy Variables

In [91]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [96]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


这abc变成三列，用1标记

In [97]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


列上面加上前缀key

In [98]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


与其他类合并

In [99]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


如果一行属于多个类别

In [100]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\anaconda\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [107]:
all_genres = []
for x in movies.genres:
    # 列表加入列表
    all_genres.extend(x.split('|'))
   
genres = pd.unique(all_genres) # 取唯一值

In [103]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [108]:
movies.genres[0]


"Animation|Children's|Comedy"

In [109]:
movies.genres[0].split('|')

['Animation', "Children's", 'Comedy']

生成行数等于movies，列数等于电影类别的全为0的数组

In [110]:
zero_matrix = np.zeros((len(movies), len(genres)))
zero_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

使用刚才整理出的电影类别，作为列，数组作为值，生成DataFrame

In [111]:
dummies = pd.DataFrame(zero_matrix, columns=genres)
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


在这张表中，我们统计，每一行对应的类别

如果属于该类别，就标记为1，如果不属于，则标记为0

In [112]:
gen = movies.genres[0]
gen

"Animation|Children's|Comedy"

In [113]:
gen.split('|')

['Animation', "Children's", 'Comedy']

找到这三个值对应的列序号

In [114]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

列序号找到了，就能确定元素的位置

让该位置元素值为1

In [115]:
type(movies.genres)

pandas.core.series.Series

In [116]:
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1
dummies

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


把标记好的列加入到原来的列中

In [117]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,6,Heat (1995),Action|Crime|Thriller,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,7,Sabrina (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,8,Tom and Huck (1995),Adventure|Children's,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,9,Sudden Death (1995),Action,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,10,GoldenEye (1995),Action|Adventure|Thriller,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


查看第0行

In [118]:
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
                                ...             
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

也可以将get_dummies与cut结合使用

In [119]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.9296, 0.3164, 0.1839, 0.2046, 0.5677, 0.5955, 0.9645, 0.6532,
       0.7489, 0.6536])

In [120]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

   (0.0, 0.2]  (0.2, 0.4]  (0.4, 0.6]  (0.6, 0.8]  (0.8, 1.0]
0           0           0           0           0           1
1           0           1           0           0           0
2           1           0           0           0           0
3           0           1           0           0           0
4           0           0           1           0           0
5           0           0           1           0           0
6           0           0           0           0           1
7           0           0           0           1           0
8           0           0           0           1           0
9           0           0           0           1           0

## 7.3 字符串操作 String Manipulation

### 7.3.1 字符串对象方法 String Object Methods

拆分

In [121]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

常常和strip一起使用，用于清除空格

In [122]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

加号合并字符串

In [123]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

join方法合并字符串

In [124]:
'::'.join(pieces)

'a::b::guido'

判断

In [126]:
val

'a,b,  guido'

In [125]:
'guido' in val

True

获取索引

In [127]:
# val.index(',')
val.index('b')

2

获取索引

In [129]:
val.find('p')

-1

与index的区别是，如果该字符不存在，index会抛出一个异常，find会返回-1

In [130]:
val.index(':')

ValueError: substring not found

计数

In [131]:
val.count(',')

2

替换

In [132]:
val.replace(',', '::')

'a::b::  guido'

In [133]:
val.replace(',', '')

'ab  guido'

### 7.3.2 正则表达式 Regular Expressions

在最开始的时候讲的非常详细了，这里就过一下

导入，分割，跟上面对字符串的操作很像，只不过是在re下的函数，使用这则表达式的规则

In [7]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

可以把写好的正则表达式规则存下来

之后重复只用

像一个函数一样

In [8]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

获得匹配正则表达式内容的列表

In [9]:
regex.findall(text)

['    ', '\t ', '  \t']

所有空格都被匹配出来

可以识别大部分邮件地址的正则表达式

In [10]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

findall返回的是列表

In [11]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search返回第一个匹配到的

记录的是其在文本中的起始位置与结束位置

In [12]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [13]:
text[m.start():m.end()]

'dave@google.com'

match只匹配开头，没有匹配到，返回None

In [14]:
print(regex.match(text))

None


替换

In [15]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



用括号将正则表达式包起来，返回的就是分开的元组

In [16]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [17]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [18]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

替换时，\1表示第一个组，\2表示第二个组

In [19]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### 7.3.3 向量化字符串函数 Vectorized String Functions in pandas

In [20]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

数据中含有缺失值，使用正则表达式很不方便

In [21]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

Series的str属性，可以跳过NA值

如下面判断字符串是否含有gmail

In [22]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

可以使用正则表达式的规则

In [23]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [24]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [25]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

判断后的结果，对对象按索引进行查找

In [28]:
data.str.get(1)

Dave       a
Steve      t
Rob        o
Wes      NaN
dtype: object

In [29]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [159]:
matches.str[0]

Dave    NaN
Steve   NaN
Rob     NaN
Wes     NaN
dtype: float64

切片

In [160]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [161]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

第七章完结！！！！

还是学到了很多技巧啊